In [538]:
cd olx-ads-recommendation-dataset/

[Errno 2] No such file or directory: 'olx-ads-recommendation-dataset/'
/Users/ankitgupta/Desktop/olx/olx-ads-recommendation-dataset


In [539]:
import pandas as pd
import numpy as np

import ast

In [540]:
test = pd.read_csv('user_messages_test.csv')
train = pd.read_csv('user_messages.csv')
ads = pd.read_csv('ads_data.csv')
user= pd.read_csv('user_data.csv')

In [541]:
user.columns

Index([u'event_time', u'user_id', u'event', u'channel', u'user_lat',
       u'user_long', u'origin', u'ad_id', u'images_count', u'ad_impressions',
       u'ad_views', u'ad_messages'],
      dtype='object')

In [542]:
user_count =user.user_id.value_counts().reset_index()
user_count.columns = ['user_id','user_count']

In [543]:
user_ad_count = user[['user_id','ad_id','origin']].groupby(['user_id','ad_id']).agg('count').reset_index()
user_ad_count.columns = ['user_id','ad_id','usr_ad_count']

In [544]:
user_ad_unique = user[['user_id','ad_id']].groupby(['user_id']).ad_id.apply(lambda x: len(x.unique())).reset_index()
user_ad_unique.columns = ['user_id','usr_ad_unique_count']

In [545]:
"""
k = user_ad_unique.merge(user_ad_count,on='user_id',how ='left')
k = k.merge(user_count,on='user_id',how ='left')
k
"""

"\nk = user_ad_unique.merge(user_ad_count,on='user_id',how ='left')\nk = k.merge(user_count,on='user_id',how ='left')\nk\n"

In [546]:
from collections import  defaultdict
dt = defaultdict(int)
count = defaultdict(int)
def f1(x):
    lst = ast.literal_eval(x)
    l = len(lst)
    
    for enu,i in enumerate(lst):
        dt[i] = dt[i]+ l*1.0/(enu+1)
        count[i] += 1
cool =train['ads'].apply(f1)

In [547]:
ads_score = pd.DataFrame({'ad_id':dt.keys(),'score':dt.values()})
ads_count = pd.DataFrame({'ad_id':count.keys(),'cou':count.values()})

In [548]:
ads_user = user[['ad_id']].merge(ads[['ad_id','category_id']],how = 'left',on= 'ad_id')

In [549]:
ads_count_in_user = ads_user.groupby('ad_id').agg('count').reset_index()
ads_count_in_user.columns = ['ad_id','ad_count']
ads_user = ads_user.merge(ads_count_in_user,how = 'left',on = 'ad_id')
ads_user  = ads_user.sort_values(['category_id','ad_count'],ascending=False)
ads_user = ads_user.drop_duplicates()
top_10_add = ads_user.groupby('category_id')['ad_id'].apply(lambda x: x.tolist()[0:10]).reset_index()
#top_10_add = top_10_add.set_index('category_id')['ad_id'].to_dict()#

In [550]:
user_ads_30 = user.groupby(['user_id'])['ad_id'].apply(lambda x: np.unique(x.tolist())).reset_index()

In [551]:
user = user.merge(ads[['ad_id','category_id']],how = 'left',on= 'ad_id')

In [552]:
user.columns

Index([u'event_time', u'user_id', u'event', u'channel', u'user_lat',
       u'user_long', u'origin', u'ad_id', u'images_count', u'ad_impressions',
       u'ad_views', u'ad_messages', u'category_id'],
      dtype='object')

In [553]:
user_ads_30_cat = user.groupby(['user_id','category_id'])['ad_id'].apply(lambda x: np.unique(x.tolist())).reset_index()

In [554]:
k = []
us = []
ca = []
for enu in range(0,user_ads_30_cat.shape[0]):
    i = user_ads_30_cat.ad_id.values[enu]
    user = user_ads_30_cat.user_id[enu]
    cat  = user_ads_30_cat.category_id[enu]
    #a = ast.literal_eval(i)
    for j in i:
        us.append(user)
        ca.append(cat)
        k.append(j)

In [555]:
train_all = pd.DataFrame({'user_id':us,'category_id':ca,'ad_id':k})
train_all = train_all.drop_duplicates()
train_all['target'] = 0

In [556]:
import ast
k = []
us = []
ca = []
for enu in range(0,train.shape[0]):
    i = train.ads.values[enu]
    user = train.user_id[enu]
    cat  = train.category_id[enu]
    a = ast.literal_eval(i)
    for j in a:
        us.append(user)
        ca.append(cat)
        k.append(j)

In [557]:
train_1 = pd.DataFrame({'user_id':us,'category_id':ca,'ad_id':k})
train_1 = train_1.drop_duplicates()
train_1['target'] = 1

In [558]:
train_final = train_1.merge(train_all,on = [ 'user_id','category_id','ad_id'],how = 'outer')

In [559]:
train_final.target_x.unique()

array([  1.,  nan])

In [560]:
train_final.loc[train_final.target_x ==1,'taget'] = 1
train_final.loc[pd.isnull(train_final.target_x) & (train_final.target_y == 0),'taget'] = 0

In [561]:
test_final= test.merge(train_all, on = ['user_id','category_id'], how = 'left')

In [562]:
test_final  = test_final.drop('target',axis =1)

In [563]:
test_final.replace(np.nan,-1,inplace= True)

In [564]:
test_no_infor = test_final.loc[test_final.ad_id == -1,['user_id','category_id']]


In [565]:
test_no_infor = test_no_infor.merge(top_10_add,on = 'category_id',how = 'left')

In [566]:

import ast
k = []
us = []
ca = []
for enu in range(0,test_no_infor.shape[0]):
    i = test_no_infor.ad_id.values[enu]
    user = test_no_infor.user_id[enu]
    cat  = test_no_infor.category_id[enu]
    #a = ast.literal_eval(i)
    for j in i:
        us.append(user)
        ca.append(cat)
        k.append(j)

In [567]:
test_no_infor = pd.DataFrame({'user_id':us,'category_id':ca,'ad_id':k})
test_no_infor = test_no_infor.drop_duplicates()


In [568]:
#test_final = test_final.loc[test_final.ad_id != -1,['user_id','category_id','ad_id']]


In [569]:
test_final.columns

Index([u'user_id', u'category_id', u'ad_id'], dtype='object')

In [570]:
test_final = pd.concat((test_final[['ad_id','category_id','user_id']],test_no_infor[['ad_id','category_id','user_id']]))
test_final = test_final.loc[test_final.ad_id != -1]
test_final = test_final.drop_duplicates()


In [571]:
few = test_final.groupby(['user_id','category_id']).agg('count').reset_index()
few.columns = ['user_id','category_id','usr_cat_count']
test_final = test_final.merge(few, on = ['user_id','category_id'], how ='left')

# cool 

In [572]:
test_no_infor = test_final.loc[test_final.usr_cat_count<=40,['user_id','category_id']]

In [573]:
test_no_infor = test_no_infor.merge(top_10_add,on = 'category_id',how = 'left')

In [574]:
import ast
k = []
us = []
ca = []
for enu in range(0,test_no_infor.shape[0]):
    i = test_no_infor.ad_id.values[enu]
    user = test_no_infor.user_id[enu]
    cat  = test_no_infor.category_id[enu]
    #a = ast.literal_eval(i)
    for j in i:
        us.append(user)
        ca.append(cat)
        k.append(j)
test_no_infor = pd.DataFrame({'user_id':us,'category_id':ca,'ad_id':k})
test_no_infor = test_no_infor.drop_duplicates()


In [575]:
#test_final = pd.concat((test_final[['ad_id','category_id','user_id']],test_no_infor[['ad_id','category_id','user_id']]))
#test_final = test_final.loc[test_final.ad_id != -1]


test_no_infor = test_no_infor.merge(test_final,on=['user_id','category_id','ad_id'], how ='left')



In [576]:
test_no_infor = test_no_infor.loc[pd.isnull(test_no_infor.usr_cat_count),['user_id','category_id','ad_id']]

In [577]:
test_final = test_final[['user_id','category_id','ad_id']]

In [578]:

train_final = train_final.drop(['target_y','target_x'],axis=1)

In [579]:
ads = pd.read_csv('ads_data.csv')
user= pd.read_csv('user_data.csv')

In [580]:
train_final = train_final.merge(user,on = ['user_id','ad_id'] ,how = 'left')
test_final = test_final.merge(user,on = ['user_id','ad_id'] ,how = 'left')

test_no_infor = test_no_infor.merge(user,on = ['user_id','ad_id'] ,how = 'left')



In [581]:

train_final['event_time'] = pd.to_datetime(train_final.event_time)
test_final['event_time'] = pd.to_datetime(test_final.event_time)
test_no_infor['event_time'] = pd.to_datetime(test_no_infor.event_time)

train_final['year'] = train_final.event_time.dt.year
train_final['month'] = train_final.event_time.dt.month
train_final['dayofyear'] = train_final.event_time.dt.dayofyear
train_final['dayofweek'] = train_final.event_time.dt.dayofweek
train_final['day'] = train_final.event_time.dt.day
test_final['year'] = test_final.event_time.dt.year
test_final['month'] = test_final.event_time.dt.month
test_final['dayofyear'] = test_final.event_time.dt.dayofyear
test_final['dayofweek'] = test_final.event_time.dt.dayofweek
test_final['day'] = test_final.event_time.dt.day

test_no_infor['year'] = test_no_infor.event_time.dt.year
test_no_infor['month'] = test_no_infor.event_time.dt.month
test_no_infor['dayofyear'] = test_no_infor.event_time.dt.dayofyear
test_no_infor['dayofweek'] = test_no_infor.event_time.dt.dayofweek
test_no_infor['day'] = test_no_infor.event_time.dt.day





In [582]:
train_final = train_final.merge(ads,how='left',on=['ad_id', 'category_id'])
test_final = test_final.merge(ads,how='left',on=['ad_id', 'category_id'])

test_no_infor = test_no_infor.merge(ads,how='left',on=['ad_id', 'category_id'])



# lets see

In [583]:
#train_final = train_final.merge(ads_score,how='left',on=['ad_id'])
#test_final = test_final.merge(ads_score,how='left',on=['ad_id'])


train_final = train_final.merge(user_count,how='left',on=['user_id'])
test_final = test_final.merge(user_count,how='left',on=['user_id'])
test_no_infor = test_no_infor.merge(user_count,how='left',on=['user_id'])

train_final = train_final.merge(user_ad_count,how='left',on=['user_id','ad_id'])
test_final = test_final.merge(user_ad_count,how='left',on=['user_id','ad_id'])
test_no_infor = test_no_infor.merge(user_ad_count,how='left',on=['user_id','ad_id'])


train_final = train_final.merge(user_ad_unique,how='left',on=['user_id'])
test_final = test_final.merge(user_ad_unique,how='left',on=['user_id'])
test_no_infor = test_no_infor.merge(user_ad_unique,how='left',on=['user_id'])



In [584]:
train_final = train_final.drop(['creation_time','lat', 'long','channel', u'user_lat', u'user_long','title', 'description'],axis =1)
test_final = test_final.drop(['creation_time','lat', 'long','channel', u'user_lat', u'user_long','title', 'description'],axis =1)
test_no_infor = test_no_infor.drop(['creation_time','lat', 'long','channel', u'user_lat', u'user_long','title', 'description'],axis =1)



#train_final = train_final.drop(['title', 'description'],axis =1)
#test_final = test_final.drop(['title', 'description'],axis =1)


In [585]:
test_final['enabled'].replace(np.nan,1,inplace = True)

In [586]:
from sklearn.preprocessing import  LabelEncoder
text_columns = []
for f in test_final.columns:
    if train_final[f].dtype== 'object':
        text_columns.append(f)            
        lbl = LabelEncoder()
        lbl.fit(list(train_final[f].values) + list(test_final[f].values)+ list(test_no_infor[f].values))
        train_final[f] = lbl.transform(list(train_final[f].values))
        test_final[f] = lbl.transform(list(test_final[f].values))
        try:
            test_no_infor[f] = lbl.transform(list(test_no_infor[f].values))
        except:
            continue
        

In [587]:
target = train_final.taget.values
test_f = test_final.copy()
test_no_infor_c = test_no_infor.copy()

train_final = train_final.drop(['taget','event_time','user_id', 'category_id', 'ad_id'],axis =1)
test_final = test_final.drop(['event_time','user_id', 'category_id', 'ad_id'],axis =1)


test_no_infor = test_no_infor.drop(['event_time','user_id', 'category_id', 'ad_id'],axis =1)


In [588]:
train_final.columns

Index([u'event', u'origin', u'images_count', u'ad_impressions', u'ad_views',
       u'ad_messages', u'year', u'month', u'dayofyear', u'dayofweek', u'day',
       u'seller_id', u'price', u'source', u'enabled', u'user_count',
       u'usr_ad_count', u'usr_ad_unique_count'],
      dtype='object')

In [589]:
"""
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=12)
train_final.replace(np.nan,-1,inplace=True)
test_final.replace(np.nan,-1,inplace=True)
model.fit(train_final,target)
"""

'\nfrom sklearn.ensemble import RandomForestClassifier\nmodel = RandomForestClassifier(n_estimators=100, max_depth=12)\ntrain_final.replace(np.nan,-1,inplace=True)\ntest_final.replace(np.nan,-1,inplace=True)\nmodel.fit(train_final,target)\n'

In [590]:
train_final.replace(np.nan,-1,inplace=True)
test_final.replace(np.nan,-1,inplace=True)
test_no_infor.replace(np.nan,-1,inplace = True)


In [591]:
import xgboost as xgb
from sklearn.metrics import  roc_auc_score
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
        params = {}
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'auc'
        params["eta"] = 0.01 #0.00334
        params["min_child_weight"] = 1
        params["subsample"] = 0.8
        params["colsample_bytree"] = 0.3
        params["silent"] = 1
        params["max_depth"] = 6
        params["seed"] = seed_val
        #params["max_delta_step"] = 2
        #params["gamma"] = 0.5
        num_rounds = 1500 #2500

        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds= 500)
        else:
                xgtest = xgb.DMatrix(test_X)
                #xgtest1 = xgb.DMatrix(test_X1)
                model = xgb.train(plst, xgtrain, num_rounds)

        if feature_names:
                        create_feature_map(feature_names)
                        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
                        importance = model.get_fscore(fmap='xgb.fmap')
                        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
                        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
                        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
                        imp_df.to_csv("imp_feat.txt", index=False)

        pred_test_y = model.predict(xgtest)
        
        if test_y is not None:
                loss = roc_auc_score(test_y, pred_test_y)
                print loss
        	return pred_test_y, loss
	else:
		return pred_test_y

In [592]:
from sklearn.model_selection import train_test_split
X = np.array(train_final)


X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

#xgb_test = np.array(test_final)


In [593]:

total = pd.concat((test_final,test_no_infor))

total = np.array(total)

k = runXGB(X, target, total)
 
test_f['prob'] = k[0:test_final.shape[0]]
test_no_infor_c['prob'] = k[test_final.shape[0]:]

In [594]:
test_f.loc[test_f.enabled ==0,'prob'] =0
test_no_infor.loc[test_no_infor.enabled ==0,'prob'] =0

In [595]:
test_f = test_f[['user_id','category_id','ad_id','prob']].drop_duplicates()
test_no_infor_c = test_no_infor_c[['user_id','category_id','ad_id','prob']].drop_duplicates()

In [596]:
last = test_f.groupby(['category_id','user_id'])['ad_id'].apply(lambda x: x.tolist()).reset_index()
last1 = test_f.groupby(['category_id','user_id'])['prob'].apply(lambda x: x.tolist()).reset_index()

In [597]:
last['prob'] = last1['prob'] 

In [598]:
def fun1(x):
    cool = [2816336]
    val = []
    prob =  x[1]
    ad_id = x[0]
    cat = x[2]
    a = np.argsort(prob)[-10:][::-1]
        
    for i in a:
        #print prob[i]
        #if prob[i] >.005:
        val.append(int(ad_id[i]))
    return val
    
last['ads'] = last[['ad_id','prob','category_id']].apply(fun1,axis =1)

# for less recommended ad_id

In [599]:
test_no_infor_c.columns

Index([u'user_id', u'category_id', u'ad_id', u'prob'], dtype='object')

In [600]:
last[['user_id','category_id','ads']].to_csv('same.csv',index = False)

In [601]:
last_less = test_no_infor_c.groupby(['category_id','user_id'])['ad_id'].apply(lambda x: x.tolist()).reset_index()
last1_less = test_no_infor_c.groupby(['category_id','user_id'])['prob'].apply(lambda x: x.tolist()).reset_index()
last_less['prob'] = last1_less['prob']
last_less['ads'] = last_less[['ad_id','prob','category_id']].apply(fun1,axis =1)

In [602]:
last = last[['category_id','user_id','ads']].merge(last_less[['category_id','user_id','ads']], on=['category_id','user_id'] , how ='left')

In [603]:
dt = ads.set_index('ad_id')['enabled'].to_dict()

In [604]:
def f1(x):
    x0 = x[0]
    x1 = x[1]
    if len(x[0])<10:
        for i in range(0,(10-len(x[0]))):
            #print len(x[0]),i,x1[i]
            x0.append(x1[i])
    return x0
                
last['ads'] = last[['ads_x', 'ads_y']].apply(f1,axis =1)
last[['user_id','category_id','ads']].to_csv('same.csv',index = False)





In [605]:
def f1(x):
    x0 = x[0]
    x1 = x[1]
    total = []   
    
    for i in x0:
        #print i,dt[i]
        if dt[i]==1:
            total.append(i)
    l = len(total)  
    #print len(x0),'\n',total
    if l ==10:
        return total
    #print x1,'1'
    if x1:
        return total
    while (True):
        #print x1[i],i
        if len(total) ==10:
            break
        if dt[x1[i]] ==1:
            total.append(x1[i])
    return total    


In [606]:
last['ads'] = last[['ads_x', 'ads_y']].apply(f1,axis =1)
last[['user_id','category_id','ads']].to_csv('same.csv',index = False)



In [607]:

top_10_add = ads_user.groupby('category_id')['ad_id'].apply(lambda x: x.tolist()[0:100]).reset_index()

top_10_add = top_10_add.set_index('category_id')['ad_id'].to_dict()

In [608]:
def f1(x):
    
    x0 = x[0]
    x1 = x[1]
    k = len(x1)
    if len(x1)<10:
        #print 'asas'
        for i in top_10_add[x0]:
            #print 'lll',top_10_add[x0]
            if (i not in x1) & (dt[i]==1):
                x1.append(i)
                if len(x1) ==10:
                    break
    #print k,len(x1)
    
    return x1[:10]

last['ads'] = last[['category_id','ads']].apply(f1,axis =1)


In [609]:
last[['user_id','category_id','ads']].to_csv('same1.csv',index = False)


In [610]:
last

,category_id,user_id,ads_x,ads_y,ads
0,362,32,"[2155129, 1151273, 1917466, 430569, 2451503, 1...",NaN,"[2155129, 1151273, 1917466, 430569, 2451503, 1..."
1,362,42,"[2684942, 1642256, 2230578, 2023961, 2680584, ...",NaN,"[2684942, 1642256, 2230578, 2023961, 2680584, ..."
2,362,66,"[1635665, 1635665, 2833390, 2833390, 2833390, ...","[2853312, 1704736, 2598159, 2836886, 2560201, ...","[1635665, 1635665, 2833390, 2833390, 2833390, ..."
3,362,182,"[2297932, 1524344, 219720, 2836886, 2560201, 1...","[2836886, 2560201, 1704736, 2757104, 2806018, ...","[2297932, 1524344, 2836886, 2560201, 1704736, ..."
4,362,191,"[1937355, 1704736, 2806363, 2377472, 5073, 384...","[2853312, 2836886, 2598159, 2560201, 2806018, ...","[1937355, 1704736, 2806363, 2377472, 5073, 384..."
5,362,196,"[2441605, 1101916, 2617331, 1509216, 563696, 3...","[2853312, 1704736, 2598159, 2836886, 2560201, ...","[2441605, 1101916, 2617331, 1509216, 563696, 2..."
6,362,205,"[2800593, 1958684, 2155129, 1958684, 2376286, ...",NaN,"[2800593, 1958684, 2155129, 1958684, 2376286, ..."
7,362,253,"[1804380, 378873, 1712278, 2606976, 2179926, 2...","[1704736, 2853312, 2836886, 2560201, 2806018, ...","[1804380, 378873, 1712278, 2606976, 2179926, 2..."
8,362,269,"[2776180, 1584381, 1275454, 2776180, 1584381, ...","[1704736, 2836886, 2560201, 2598159, 2806018, ...","[2776180, 1584381, 1275454, 2776180, 1584381, ..."
9,362,284,"[2844620, 2521057, 2836886, 2560201, 1704736, ...","[2836886, 2560201, 1704736, 2806018, 2598159, ...","[2844620, 2836886, 2560201, 1704736, 2806018, ..."
